In [1]:
import re
from copy import copy

instructions = []
rule = re.compile("(?P<switch>on|off) x=(?P<x1>-?\d+)..(?P<x2>-?\d+),y=(?P<y1>-?\d+)..(?P<y2>-?\d+),z=(?P<z1>-?\d+)..(?P<z2>-?\d+).*")
with open("input.txt", "r") as f:
    lines = f.read().splitlines()
    for line in lines:
        gd = rule.search(line).groupdict()
        instructions.append({"switch": gd["switch"]=="on",
                             "x_range": (int(gd["x1"]),int(gd["x2"])),
                             "y_range": (int(gd["y1"]),int(gd["y2"])),
                             "z_range": (int(gd["z1"]),int(gd["z2"]))})
        
def _in_boundaries(r):
    return -50<=r[0]<=50 and -50<=r[1]<=50

def in_boundaries(inst):
    return _in_boundaries(inst["x_range"]) and _in_boundaries(inst["y_range"]) and _in_boundaries(inst["z_range"])
  
def is_disjoint(inst1, inst2):
    return any([
        inst1["x_range"][1]<inst2["x_range"][0] or inst1["x_range"][0]>inst2["x_range"][1],
        inst1["y_range"][1]<inst2["y_range"][0] or inst1["y_range"][0]>inst2["y_range"][1],
        inst1["z_range"][1]<inst2["z_range"][0] or inst1["z_range"][0]>inst2["z_range"][1]])
    
def equals(inst1, inst2):
    return all([
        inst1["x_range"]==inst2["x_range"],
        inst1["y_range"]==inst2["y_range"],
        inst1["z_range"]==inst2["z_range"]])

def volume(inst):
    return (inst["x_range"][1]-inst["x_range"][0]+1)*(inst["y_range"][1]-inst["y_range"][0]+1)*(inst["z_range"][1]-inst["z_range"][0]+1)

def intersection(inst1, inst2):
    if is_disjoint(inst1, inst2):
        return None
    else:
        return dict(
            x_range=(max(inst1["x_range"][0],inst2["x_range"][0]), min(inst1["x_range"][1],inst2["x_range"][1])),
            y_range=(max(inst1["y_range"][0],inst2["y_range"][0]), min(inst1["y_range"][1],inst2["y_range"][1])),
            z_range=(max(inst1["z_range"][0],inst2["z_range"][0]), min(inst1["z_range"][1],inst2["z_range"][1])),
        )

def make_cuboids(xs, ys, zs, intersec):
    cuboids = []
    for x in xs:
        for y in ys:
            for z in zs:
                c = dict(x_range = x,
                y_range = y,
                z_range = z)
                if not equals(c, intersec):
                    cuboids.append(c)
    return cuboids
        
def _split_axis(c_range, intersec_range):
    xs = []
    next_x = c_range[0]
    if c_range[0] < intersec_range[0] <= c_range[1]:
        xs.append((next_x,intersec_range[0]-1))
        next_x = intersec_range[0]
    if c_range[0] <= intersec_range[1] < c_range[1]:
        xs.append((next_x, intersec_range[1]))
        next_x = intersec_range[1]+1
    xs.append((next_x, c_range[1]))
    return xs
    
def split(c, intersec):
    xs = _split_axis(c["x_range"], intersec["x_range"])
    ys = _split_axis(c["y_range"], intersec["y_range"])
    zs = _split_axis(c["z_range"], intersec["z_range"])
    return make_cuboids(xs, ys, zs, intersec)
                
def perform_instructions(instructions, limit_range):
    on_cuboids = []
    for inst in instructions:
        if not limit_range or in_boundaries(inst): 
            initial_on_cuboids = copy(on_cuboids)   
            for c in initial_on_cuboids:
                intersec = intersection(inst, c)
                if intersec is not None:
                    new_cuboids = split(c, intersec)
                    on_cuboids.remove(c)
                    on_cuboids += new_cuboids
            if inst["switch"]==True:
                on_cuboids.append(inst)
    return on_cuboids

cuboids_1 = perform_instructions(instructions, True)
answer_1 = sum([volume(c) for c in cuboids_1])

cuboids_2 = perform_instructions(instructions, False)
answer_2 = sum([volume(c) for c in cuboids_2])

print(f"answer_1: {answer_1}")
print(f"answer_2: {answer_2}")    
     

answer_1: 583636
answer_2: 1294137045134837
